In [38]:
import pandas as pd

In [3]:
clinical_data = pd.read_csv("./data/clinical_annotations.tsv", sep='\t')

In [45]:
new_clinical_data = pd.read_csv("./data/PDC_clinical_manifest_07042024_120812.csv")

In [48]:
new_clinical_data["Tumor Grade"].value_counts()

Tumor Grade
G2              145
Not Reported    104
G3               40
G1               13
G4                1
Name: count, dtype: int64

In [42]:
new_clinical_data.loc[new_clinical_data["Tumor Grade"]=="G1"]["Cases Submitter ID"]

8      C3L-00928
31     C3L-00622
49     C3N-02971
57     C3L-03394
86     C3L-00599
97     C3N-02295
133    C3L-02604
136    C3L-03632
162    C3N-03190
Name: Cases Submitter ID, dtype: object

In [4]:
clinical_data["tumor_grade"].value_counts()

tumor_grade
G2              614
G3              294
Not Reported    176
G1              151
GX               26
G4               26
'--              22
Unknown           4
High Grade        1
Name: count, dtype: int64

In [36]:
#Sample 57 patients with balanced labels
reference_patients = set(pd.read_csv("data/Metadata_Report_CPTAC-PDA_2023_07_14.csv")["PatientID"])

#print(reference_patients)

g1_patients = clinical_data.loc[clinical_data["tumor_grade"] == "G1"]["case_submitter_id"]

print(len(g1_patients[g1_patients.isin(reference_patients)])) #Only 5 patients with clinical annotations with g1 cancer grade 
selected_g1_patients = g1_patients[g1_patients.isin(reference_patients)].sample(n=5, replace = False)


g2_patients = clinical_data.loc[clinical_data["tumor_grade"] == "G2"]["case_submitter_id"]
print(len(g2_patients[g2_patients.isin(reference_patients)]))

selected_g2_patients = g2_patients[g2_patients.isin(reference_patients)].sample(n=33, replace = False)

g3_patients = clinical_data.loc[clinical_data["tumor_grade"] == "G3"]["case_submitter_id"]
print(len(g3_patients[g3_patients.isin(reference_patients)]))

selected_g3_patients = g3_patients[g3_patients.isin(reference_patients)].sample(n=19, replace = False)

5
49
19


In [37]:
with open("data/sampled_patients.txt", "a") as file:
    file.write(',\n'.join(selected_g1_patients.astype(str))+',\n')
    file.write(',\n'.join(selected_g2_patients.astype(str))+',\n')
    file.write(',\n'.join(selected_g3_patients.astype(str))+',\n')